## 노이즈가 추가되지 않은 텍스트 filtering
- 특수 문자 비율이 높으면 노이즈가 추가되었을 가능성이 있음
- 한글 및 알파벳, 숫자, 공백 제외 -> threshold 임의 설정
- 추가적으로 한자도 제외하면 좋을 듯

In [38]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('../data/train/train.csv')

In [4]:
# 특수 문자의 비율을 계산하여 노이즈로 간주
def is_noisy_text(text, threshold=0.06):
    special_char_ratio = sum(not (c.isalnum() or c.isspace()) for c in text) / len(text)
    return special_char_ratio > threshold

# 노이즈가 없는 텍스트 필터링
noise_free_texts = train_df[~train_df['text'].apply(is_noisy_text)]

In [5]:
noise_free_texts

,ID,text,target
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
6,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
...,...,...,...
2794,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


___

## 한자는 특수문자 x + 영어 비율 고려
- 한글, 한자 및 알파벳과 숫자, 공백은 특수문자로 간주하지 않음
- 영어 비율이 일정 threshold 이상이면 걸러내는 코드

In [4]:
train_df = pd.read_csv('../data/train/train.csv')

In [14]:
import re

# 한글, 한자, 숫자, 공백을 제외한 특수 문자를 비율로 계산
def is_noisy_text(text, special_char_threshold=0.1, english_ratio_threshold=0.1):
    # 특수 문자를 제외한 비율 계산을 위한 패턴 (한글, 한자, 알파벳, 숫자, 공백 허용)
    special_char_pattern = re.compile(r'[\u4e00-\u9fff\U00020000-\U0002a6df\u3131-\u318e\uac00-\ud7a3a-zA-Z0-9\s]')
    # 영어 문자의 비율 계산
    english_char_pattern = re.compile(r'[a-zA-Z]')
    
    # 특수 문자의 비율 계산
    special_char_ratio = sum(not special_char_pattern.match(c) for c in text) / len(text)
    
    # 영어 문자의 비율 계산
    english_char_ratio = sum(english_char_pattern.match(c) is not None for c in text) / len(text)
    
    # 특수 문자 비율 또는 영어 비율 중 하나가 threshold를 초과하면 노이즈로 간주
    return special_char_ratio > special_char_threshold or english_char_ratio > english_ratio_threshold

# 노이즈가 없는 텍스트 필터링
noise_free_texts = train_df[~train_df['text'].apply(is_noisy_text)]


In [15]:
noise_free_texts

,ID,text,target
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
11,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1
...,...,...,...
2794,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [16]:
noise_free_texts.to_csv('noise_free_eng_threshold_0.1.csv')

In [17]:
# 각 target별 개수와 비율 구하기
target_counts = noise_free_texts['target'].value_counts()
target_ratios = noise_free_texts['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
5         178  0.165581
1         159  0.147907
6         153  0.142326
4         153  0.142326
0         150  0.139535
3         142  0.132093
2         140  0.130233


In [ ]:
noisy_texts = train_df[train_df['text'].apply(is_noisy_text)]
noisy_texts

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
6,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1
...,...,...,...
2788,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0
2789,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",2
2790,ynat-v1_train_02790,프로농구 개막 ③ 헤인즈의 1만점·양동근의 900스틸…...,3
2792,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",1


: 

___

### exp

In [1]:
class_names = ['생활문화', '스포츠', '정치', '사회', 'IT과학', '경제', '세계']

# 클래스와 인덱스 매핑
class_to_idx = {
    '생활문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    'IT과학': 4,
    '경제': 5,
    '세계': 6
}

In [2]:
import torch
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader

import cleanlab
from cleanlab.filter import find_label_issues

In [46]:
train_df = pd.read_csv('../data/train/noise_free_threshold_0.08.csv')
preliminary_labels = train_df['target']

In [49]:
train_df

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1
...,...,...,...
1242,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1243,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1244,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
1245,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [50]:
output_df = pd.read_csv('./output/noise_free_output_with_prob.csv')
model_pred_probs = output_df[[f'prob_{i}' for i in range(7)]].values

In [52]:
output_df = output_df.drop(columns=['Unnamed: 0'])

In [53]:
# 초기 라벨과 모델 예측 확률을 통해 라벨 오류 의심 샘플 찾기
label_issues = find_label_issues(
    labels=preliminary_labels,       # 초기 라벨
    pred_probs=model_pred_probs,     # 모델의 예측 확률
)

In [54]:
# 라벨 오류 가능성이 있는 샘플의 인덱스만 추출
label_issue_indices = np.where(label_issues)[0]

# 오류 샘플에서 모델이 예측한 라벨 보기
corrected_labels = [model_pred_probs[idx].argmax() for idx in label_issue_indices]

# 라벨을 모델의 예측 라벨로 교체
train_df.loc[label_issue_indices, 'target'] = corrected_labels

In [56]:
train_df.to_csv('noise_free_train.csv')

In [59]:
train_df

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,3
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,2
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,2
...,...,...,...
1242,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1243,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1244,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
1245,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,2


In [60]:
# 각 target별 개수와 비율 구하기
target_counts = train_df['target'].value_counts()
target_ratios = train_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
5         387  0.310345
2         220  0.176423
1         195  0.156375
6         157  0.125902
3         132  0.105854
4          78  0.062550
0          78  0.062550


## Hierarchical Denoising

In [1]:
import pandas as pd

train_df = pd.read_csv('../train_noisy.csv')
train_df

,ID,text,target,target_name,re_text
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,IT과학,"정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)"
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,사회,"경찰, '국회 불법 로비' 한어총 회장 등 20명 송치"
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,정치,"김정은 ""자주통일 새시대 열어나가야"""
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,세계,"軍 ""美대선 앞두고 北 무수단 발사 대비해 감시 강화"""
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1,스포츠,프로야구 롯데-KIA 광주 경기 우천취소
...,...,...,...,...,...
1582,ynat-v1_train_02787,13이 노바 라이n2·미J어패0 T3 10 결G상품% *,6,세계,13일 노바 라이프미주 3개 품목 10% 할인 상품 출시
1583,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0,생활문화,남원소식 춘향단장 모집
1584,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",2,정치,"이총리, 세출 예산 완전 집행 못해 할 일 꽤 남아"
1585,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",1,스포츠,높은 취업률...선수 몸값 상승 논란


In [11]:
import re

# 한글, 한자, 숫자, 공백을 제외한 특수 문자를 비율로 계산
def basic_text(text, special_char_threshold=0.25):
    # 특수 문자를 제외한 비율 계산을 위한 패턴 (한글, 한자, 알파벳, 숫자, 공백 허용)
    special_char_pattern = re.compile(r'[\u4e00-\u9fff\U00020000-\U0002a6df\u3131-\u318e\uac00-\ud7a30-9\s]')

    # 특수 문자의 비율 계산
    special_char_ratio = sum(not special_char_pattern.match(c) for c in text) / len(text)
        
    return special_char_ratio < special_char_threshold

In [12]:
# threshold: 0.25
basic_texts = train_df[train_df['text'].apply(basic_text)]
basic_texts.to_csv('basic_noisy_25.csv')

In [ ]:
# 각 target별 개수와 비율 구하기
target_counts = basic_texts['target'].value_counts()
target_ratios = basic_texts['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
3          50  0.187266
6          46  0.172285
2          43  0.161049
5          39  0.146067
0          37  0.138577
1          30  0.112360
4          22  0.082397


: 

In [ ]:
def normal_text(text, special_char_threshold=0.5):
    # 특수 문자를 제외한 비율 계산을 위한 패턴 (한글, 한자, 알파벳, 숫자, 공백 허용)
    special_char_pattern = re.compile(r'[\u4e00-\u9fff\U00020000-\U0002a6df\u3131-\u318e\uac00-\ud7a30-9\s]')

    # 특수 문자의 비율 계산
    special_char_ratio = sum(not special_char_pattern.match(c) for c in text) / len(text)
        
    return special_char_ratio < special_char_threshold

In [14]:
rest_texts = train_df.loc[~train_df.index.isin(basic_texts.index)]

In [15]:
# threshold: 0.45
normal_texts = rest_texts[rest_texts['text'].apply(normal_text)]
normal_texts

,ID,text,target,target_name,re_text
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,사회,"경찰, '국회 불법 로비' 한어총 회장 등 20명 송치"
5,ynat-v1_train_00010,oi 매력 R모h츠a열#w3약 >l·주가 고Q/진,5,경제,오일 매력적인 모델들이 열을 올리며 약 3억 주가 고평가되고 있다
6,ynat-v1_train_00013,아이`XSI수리0* b대`…맥3 디dF레< 41/',4,IT과학,아이들의 상상력을 수리하는 마법 같은 시간...맥락 없는 대화 41분
9,ynat-v1_train_00021,KIA I수단·팬nI께하는:호kQ4M족 한마5 S최,1,스포츠,KIA 수단·팬과 함께하는 희망:호국훈련 4차 모임 한마당 5회
10,ynat-v1_train_00022,현N차sJ <e 임원6늘려…~세z 리g (보j 육y,3,사회,현대로템 임원 6명 늘려...세제 개편 (보도)
...,...,...,...,...,...
1565,ynat-v1_train_02759,sh기하는 이p미w대표와 >i하)/QQ표,2,정치,시진핑 중국 대표와 인도 총리 회담
1566,ynat-v1_train_02763,12월)ko% 손흥H =?u시즌보D T른jEu 가속도,1,스포츠,"12월 손흥찬 감독, 올해 시즌보다 내년 시즌이 더 기대된다고 말했다"
1567,ynat-v1_train_02764,美|Tb3{주+대서(K격…8명 부상…1명{중N1보,6,세계,美 텍사스 대규모 폭발 사고로 8명 부상...1명 중태
1569,ynat-v1_train_02766,"Q산U 림자금\ I,SC다…Y감원장 b합YM시스템_구6",5,경제,"금융감독원장, '산은 자금 유출' SC제일은행 조사 착수"


In [16]:
# basic_texts와 normal_texts의 인덱스를 모두 합침
excluded_indices = basic_texts.index.union(normal_texts.index)

# excluded_indices에 포함되지 않은 인덱스만 남겨서 hard_texts 생성
hard_texts = train_df.loc[~train_df.index.isin(excluded_indices)]
hard_texts

,ID,text,target,target_name,re_text
15,ynat-v1_train_00029,"p-축구M인천 ,qN 35c운드 r&Q{영`",1,스포츠,"축구, 인천 유나이티드 35라운드 원정 경기"
20,ynat-v1_train_00037,"dcST C, 생산 인공광Fy에d""기보다BS에*!6활ht""y@적",4,IT과학,"정부, '태양광 발전 인공광에 생산 효율보다 비용 절감이 더 중요'"
24,ynat-v1_train_00045,x' 유^ 의H%eprM못gz고r사표…특별1LX제 흔들,3,사회,"검찰, '유력 후보 지지 선언' 표밭갈이...특별법 제정 흔들"
27,ynat-v1_train_00048,"Vq)Z!\자o구 김정Re트로1$보니C.도l모르"".Qk이",1,스포츠,"베트남 구제금융 신청, 김정은 '트로이 목마' 전략 보니 정부도 모르는 상황"
28,ynat-v1_train_00049,"인국공I{태NL년D의 bT,무jlv공u{고?진보인가",3,사회,"인천국제공항공사, 올해의 청년 혁신 아이디어 공모전 개최"
...,...,...,...,...,...
1570,ynat-v1_train_02767,청Z한 1요BM전국 vvX큰>일d>T감fcN*M세요,0,생활문화,청년층 1월 전국 일자리 큰 폭 증가
1573,ynat-v1_train_02773,"*@이RAO _zW첫 스3#U 메\트,20…최고 1L(Y원OV출}",4,IT과학,"이개호 농림축산식품부 장관, '농업 혁신' 첫 회의 개최"
1574,ynat-v1_train_02774,문답으0`#어=mX거*지T로드맵 n\내#]C,3,사회,"문 대통령, '디지털 뉴딜 로드맵' 내거치다"
1577,ynat-v1_train_02779,+클리 스마트G=<MVx 쓸Y때c가장{jZf까,4,IT과학,LG 클리커스 스마트 기기 사용할 때 가장 주의해야 할 점은 배터리 관리
